Technological Institute of the Philippines | Quezon City - Computer Engineering
--- | ---
Course Code: | CPE 313
Code Title: | Advanced Machine Learning and Deep Learning
2nd Semester | SY 2024-2025
<hr> | <hr>
<u>**Discussion 3.1** | **Performing Face Recognition**
**Name** | Belocora, John Rome
**Section** | CPE32S3
**Date Performed**: | 02/20/2025
**Date Submitted**: | 02/20/2025
**Instructor**: | Engr. Roman M. Richard

<hr>

## 1. Objectives

This activity aims to enable students to perform data preparation and face recognition on their own generated dataset.

## 2. Intended Learning Outcomes (ILOs)
After this activity, the students should be able to:
* Utilize data preparation techniques for images.
* Perform Face Recognition using multiple algorithms.
* Evaluate the performance of different algorithms.

## 3. Procedures and Outputs

### Preparing the training data

Now that we have our data, we need to load these sample pictures into our face recognition algorithms. All face recognition algorithms take two parameters in their `train()` method: an array of images and an array of labels. What do these labels represent? They are the IDs of a certain individual/face so that when face recognition is performed, we not only know the person was recognized but also who—among the many people available in our database—the person is.

To do that, we need to create a comma-separated value (CSV) file, which will contain the path to a sample picture followed by the ID of that person.

**Include a Screenshot of Your Dataset Here**

---

### Loading the data and recognizing faces

Next up, we need to load these two resources (the array of images and CSV file) into the face recognition algorithm, so it can be trained to recognize our face. To do this, we build a function that reads the CSV file and—for each line of the file—loads the image at the corresponding path into the images array and the ID into the labels array.

In [1]:
import numpy as np
import os
import errno
import sys
import cv2

def read_images(path, sz=None):
  c = 0
  X, y = [], []

  for dirname, dirnames, filenames in os.walk(path):
    for subdirname in dirnames:
      subject_path = os.path.join(dirname, subdirname)
      for filename in os.listdir(subject_path):
        try:
          if(filename == ".directory"):
            continue
          filepath = os.path.join(subject_path, filename)
          im = cv2.imread(os.path.join(subject_path, filename), cv2.IMREAD_GRAYSCALE)

          # Resize the images to the prescribed size
          if (sz is not None):
            im = cv2.resize(im, (200,200))

          X.append(np.asarray(im, dtype=np.uint8))
          y.append(c)

        except IOError as e:
          print(f"I/O Error({e.errno}): {e.strerror}")
        except:
          print("Unexpected error:", sys.exc_info()[0])
          raise
      c = c+1
  return [X, y]

dataset = read_images(r"C:\Users\John Rome Belocora\Downloads\act 7\dataset")
print(dataset)

[[array([[198, 200, 201, ..., 130, 140, 147],
       [197, 199, 200, ..., 125, 118, 113],
       [197, 198, 199, ..., 125, 126, 126],
       ...,
       [145, 144, 135, ...,  75,  76,  77],
       [138, 137, 126, ...,  76,  87,  96],
       [134, 139, 129, ...,  67,  67,  67]],
      shape=(720, 1280), dtype=uint8), array([[200, 198, 199, ..., 127, 127, 127],
       [200, 198, 198, ..., 126, 126, 126],
       [200, 199, 198, ..., 126, 125, 125],
       ...,
       [138, 140, 141, ...,  71,  75,  74],
       [136, 142, 145, ...,  72,  75,  74],
       [135, 138, 139, ...,  73,  76,  73]],
      shape=(720, 1280), dtype=uint8), array([[225, 225, 226, ..., 230, 247, 255],
       [219, 220, 222, ..., 227, 236, 242],
       [214, 215, 218, ..., 225, 223, 222],
       ...,
       [ 43,  27,   2, ..., 158, 168, 174],
       [ 42,  27,   2, ..., 157, 166, 172],
       [ 41,  26,   2, ..., 156, 165, 171]],
      shape=(2048, 1536), dtype=uint8), array([[130, 123,  95, ..., 218, 220, 224],
     

**Question: Run the function above on your generated dataset. Provide an analysis and note all the challenges you have encountered running this code.**

---

The script effectively reads and labels images but can be improved by handling empty images, optimizing performance, and providing clearer error messages. It serves as a strong foundation for facial recognition or image classification tasks. The challenges that I faced was fixing the dataset file directory and compiling the pictures on it.

### Performing Face Recognition Algorithms

Here is a sample script for testing the Face Recognition Algorithm. In this section, we're going to follow the same process but with different algorithms for face recognitions, namely:
- Eigenface Recognition
- Fisherface Recognition
- Local Binary Pattern Histograms (LBPH) Recognition

In [ ]:
import cv2
import numpy as np
import os
import sys

def read_images(path, sz=(200, 200)):
    """Reads images from dataset and resizes them."""
    X, y, label = [], [], 0
    for subdir in filter(lambda d: os.path.isdir(os.path.join(path, d)), os.listdir(path)):
        for filename in os.listdir(os.path.join(path, subdir)):
            img = cv2.imread(os.path.join(path, subdir, filename), cv2.IMREAD_GRAYSCALE)
            if img is not None:
                X.append(cv2.resize(img, sz))
                y.append(label)
        label += 1
    return np.array(X, dtype=np.uint8), np.array(y, dtype=np.int32)

def face_rec():
    names, dataset_path = ['Joms', 'Moms'], "dataset"
    if not os.path.exists(dataset_path):
        sys.exit(f"Error: Dataset path '{dataset_path}' not found!")

    X, y = read_images(dataset_path)
    if X.size == 0:
        sys.exit("Error: No images found in dataset.")

    try:
        model = cv2.face.EigenFaceRecognizer_create()
        model.train(X, y)
    except AttributeError:
        sys.exit("Error: OpenCV does not support face recognition. Install 'opencv-contrib-python'.")

    face_cascade = cv2.CascadeClassifier(r"C:\Users\John Rome Belocora\Downloads\haarcascade_frontalface_default.xml")
    camera = cv2.VideoCapture(0)
    if not camera.isOpened():
        sys.exit("Error: Could not access webcam.")

    while True:
        ret, img = camera.read()
        if not ret:
            break

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        for (x, y, w, h) in face_cascade.detectMultiScale(gray, 1.3, 5):
            roi = cv2.resize(gray[y:y+h, x:x+w], (200, 200))
            label_index, confidence = model.predict(roi)

            label = names[label_index] if label_index < len(names) and confidence < 5000 else "Unknown"
            cv2.putText(img, f"{label}, {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)

        cv2.imshow("Face Recognition", img)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    camera.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    face_rec()


**Question: Provide an analysis of the sample script for the process using the Eigenface Model. What is the sample code doing? Are you able to troubleshoot any problems encountered?**

The Eigenface Trains the model Principal Component Analysis (PCA) to identify and recognize faces. Yes.

---
Perform the remaining face recognition techniques by using the same (or modified) process from the sample code:

- `model = cv2.face.createFisherFaceRecognizer()`
- `model = cv2.face.createLBPHFaceRecognizer()`

**Question: The `predict()` method returns a two-element array. Provide your analysis of the two returned values and their important ince this application.**

## FisherFaceRecognizer

In [ ]:
def read_images(path, sz=(200, 200)):
    """Reads images from dataset and resizes them."""
    X, y, label = [], [], 0
    for subdir in filter(lambda d: os.path.isdir(os.path.join(path, d)), os.listdir(path)):
        for filename in os.listdir(os.path.join(path, subdir)):
            img = cv2.imread(os.path.join(path, subdir, filename), cv2.IMREAD_GRAYSCALE)
            if img is not None:
                X.append(cv2.resize(img, sz))
                y.append(label)
        label += 1
    return np.array(X, dtype=np.uint8), np.array(y, dtype=np.int32)

def face_rec():
    names, dataset_path = ['Joms', 'Moms'], "dataset"
    if not os.path.exists(dataset_path):
        sys.exit(f"Error: Dataset path '{dataset_path}' not found!")

    X, y = read_images(dataset_path)
    if X.size == 0:
        sys.exit("Error: No images found in dataset.")

    try:
        model = cv2.face.FisherFaceRecognizer_create()
        model.train(X, y)
    except AttributeError:
        sys.exit("Error: OpenCV does not support face recognition. Install 'opencv-contrib-python'.")

    face_cascade = cv2.CascadeClassifier(r"C:\Users\John Rome Belocora\Downloads\haarcascade_frontalface_default.xml")
    camera = cv2.VideoCapture(0)
    if not camera.isOpened():
        sys.exit("Error: Could not access webcam.")

    while True:
        ret, img = camera.read()
        if not ret:
            break

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        for (x, y, w, h) in face_cascade.detectMultiScale(gray, 1.3, 5):
            roi = cv2.resize(gray[y:y+h, x:x+w], (200, 200))
            label_index, confidence = model.predict(roi)

            label = names[label_index] if label_index < len(names) and confidence < 5000 else "Unknown"
            cv2.putText(img, f"{label}, {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)

        cv2.imshow("Face Recognition", img)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    camera.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    face_rec()


## LBPHFaceRecognizer

In [ ]:
def read_images(path, sz=(200, 200)):
    """Reads images from dataset and resizes them."""
    X, y, label = [], [], 0
    for subdir in filter(lambda d: os.path.isdir(os.path.join(path, d)), os.listdir(path)):
        for filename in os.listdir(os.path.join(path, subdir)):
            img = cv2.imread(os.path.join(path, subdir, filename), cv2.IMREAD_GRAYSCALE)
            if img is not None:
                X.append(cv2.resize(img, sz))
                y.append(label)
        label += 1
    return np.array(X, dtype=np.uint8), np.array(y, dtype=np.int32)

def face_rec():
    names, dataset_path = ['Joms', 'Moms'], "dataset"
    if not os.path.exists(dataset_path):
        sys.exit(f"Error: Dataset path '{dataset_path}' not found!")

    X, y = read_images(dataset_path)
    if X.size == 0:
        sys.exit("Error: No images found in dataset.")

    try:
        model = cv2.face.LBPHFaceRecognizer_create()
        model.train(X, y)
    except AttributeError:
        sys.exit("Error: OpenCV does not support face recognition. Install 'opencv-contrib-python'.")

    face_cascade = cv2.CascadeClassifier(r"C:\Users\John Rome Belocora\Downloads\haarcascade_frontalface_default.xml")
    camera = cv2.VideoCapture(0)
    if not camera.isOpened():
        sys.exit("Error: Could not access webcam.")

    while True:
        ret, img = camera.read()
        if not ret:
            break

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        for (x, y, w, h) in face_cascade.detectMultiScale(gray, 1.3, 5):
            roi = cv2.resize(gray[y:y+h, x:x+w], (200, 200))
            label_index, confidence = model.predict(roi)

            label = names[label_index] if label_index < len(names) and confidence < 5000 else "Unknown"
            cv2.putText(img, f"{label}, {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)

        cv2.imshow("Face Recognition", img)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    camera.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    face_rec()


## 4. Supplementary Activity

Your accomplisment of the tasks below contribute to the achievement of ILO1, ILO2, and ILO3 for this module.

---

Tasks:
1. Create a new dataset for testing, this dataset must include the following:
  - The same person/s that the model has to recognize.
  - Different person/s that the model should not recognize.
2. For each model, perform 20 tests. Document the testing performed and provide observations.
3. Conclude on the performed tests by providing your evaluation of the performance of the models.

In [ ]:
import cv2
import numpy as np
import os
import sys
import time

def read_images(path, sz=(200, 200)):
    """Reads images from dataset and resizes them."""
    X, y, label = [], [], 0
    for subdir in filter(lambda d: os.path.isdir(os.path.join(path, d)), os.listdir(path)):
        for filename in os.listdir(os.path.join(path, subdir)):
            img = cv2.imread(os.path.join(path, subdir, filename), cv2.IMREAD_GRAYSCALE)
            if img is not None:
                X.append(cv2.resize(img, sz))
                y.append(label)
        label += 1
    return np.array(X, dtype=np.uint8), np.array(y, dtype=np.int32)

def face_rec():
    names, dataset_path = ['Joms', 'Moms'], "dataset"
    if not os.path.exists(dataset_path):
        sys.exit(f"Error: Dataset path '{dataset_path}' not found!")

    X, y = read_images(dataset_path)
    if X.size == 0:
        sys.exit("Error: No images found in dataset.")

    try:
        model = cv2.face.EigenFaceRecognizer_create()
        model.train(X, y)
    except AttributeError:
        sys.exit("Error: OpenCV does not support face recognition. Install 'opencv-contrib-python'.")

    face_cascade = cv2.CascadeClassifier(r"C:\Users\John Rome Belocora\Downloads\haarcascade_frontalface_default.xml")
    camera = cv2.VideoCapture(0)
    if not camera.isOpened():
        sys.exit("Error: Could not access webcam.")

    test_count = 0 
    results = [] 

    while test_count < 20:
        ret, img = camera.read()
        if not ret:
            break

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            roi = cv2.resize(gray[y:y+h, x:x+w], (200, 200))
            label_index, confidence = model.predict(roi)

            label = names[label_index] if label_index < len(names) and confidence < 5000 else "Unknown"
            timestamp = time.strftime("%Y-%m-%d %H:%M:%S")

            # Store result
            results.append(f"Test {test_count+1}: {label}, Confidence: {confidence:.2f}, Time: {timestamp}")
            print(results[-1])

            # Display on screen
            cv2.putText(img, f"{label}, {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)

        cv2.imshow("Face Recognition", img)
        test_count += 1

        # Wait for 1 minute before the next test
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
        time.sleep(60)

    # Save results to a file
    with open("test_results.txt", "w") as f:
        f.write("\n".join(results))

    print("Testing completed. Results saved in 'test_results.txt'.")

    camera.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    face_rec()

: 

## 5. Summary, Conclusions and Lessons Learned

<hr/>

***Proprietary Clause***

*Property of the Technological Institute of the Philippines (T.I.P.). No part of the materials made and uploaded in this learning management system by T.I.P. may be copied, photographed, printed, reproduced, shared, transmitted, translated, or reduced to any electronic medium or machine-readable form, in whole or in part, without the prior consent of T.I.P.*